# Markdown → Google Docs (Meeting Notes) — Colab Notebook

This notebook authenticates to Google in Colab, creates a new Google Doc, and converts the provided markdown meeting notes into a formatted Google Doc (headings, nested bullets, checkboxes, mention styling, footer styling).


In [ ]:
# (Optional) Install dependencies (usually already available in Colab)
!pip -q install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [ ]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


In [ ]:
# Provided markdown meeting notes
MARKDOWN_NOTES = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""


In [ ]:
import re
from dataclasses import dataclass
from typing import List, Optional, Tuple

@dataclass
class Block:
    kind: str  # 'h1','h2','h3','p','bullet','hr','footer'
    text: str = ""
    level: int = 0          # for bullets
    checkbox: bool = False  # for bullets
    checked: bool = False   # for bullets

MENTION_RE = re.compile(r'@([A-Za-z0-9_\-]+)')

def parse_markdown(md: str) -> List[Block]:
    """Parse a small, meeting-notes-flavored subset of Markdown into structured blocks."""
    lines = md.replace('\t', '  ').splitlines()
    blocks: List[Block] = []
    in_footer = False

    for raw in lines:
        line = raw.rstrip('\n')
        if line.strip() == "":
            # Keep empty line as paragraph break, unless we are in footer (footer typically compact)
            if not in_footer:
                blocks.append(Block(kind="p", text=""))
            continue

        if line.strip() == "---":
            blocks.append(Block(kind="hr"))
            in_footer = True
            continue

        # Headings
        if line.startswith("# "):
            # Requirement: main title should be "Product Team Sync" as H1 style.
            # If the line contains " - " split into title and date.
            title = line[2:].strip()
            if " - " in title:
                main, rest = title.split(" - ", 1)
                blocks.append(Block(kind="h1", text=main.strip()))
                blocks.append(Block(kind="p", text=rest.strip()))
            else:
                blocks.append(Block(kind="h1", text=title))
            continue
        if line.startswith("## "):
            blocks.append(Block(kind="h2", text=line[3:].strip()))
            continue
        if line.startswith("### "):
            blocks.append(Block(kind="h3", text=line[4:].strip()))
            continue

        # Bullets + checkboxes
        m_cb = re.match(r'^(\s*)[-*]\s*\[( |x|X)\]\s+(.*)$', line)
        if m_cb:
            spaces = len(m_cb.group(1))
            level = spaces // 2
            checked = m_cb.group(2).lower() == "x"
            text = m_cb.group(3).strip()
            blocks.append(Block(kind="bullet", text=text, level=level, checkbox=True, checked=checked))
            continue

        m_b = re.match(r'^(\s*)([-*])\s+(.*)$', line)
        if m_b:
            spaces = len(m_b.group(1))
            level = spaces // 2
            text = m_b.group(3).strip()
            blocks.append(Block(kind="bullet", text=text, level=level, checkbox=False))
            continue

        # Footer lines (after ---) should be styled distinctly
        if in_footer:
            blocks.append(Block(kind="footer", text=line.strip()))
        else:
            blocks.append(Block(kind="p", text=line.strip()))

    # Compact consecutive blank paragraphs
    compact: List[Block] = []
    prev_blank = False
    for b in blocks:
        if b.kind == "p" and b.text == "":
            if prev_blank:
                continue
            prev_blank = True
        else:
            prev_blank = False
        compact.append(b)
    return compact

blocks = parse_markdown(MARKDOWN_NOTES)
blocks[:10], len(blocks)


In [ ]:
def build_doc_requests(blocks: List[Block]) -> Tuple[str, List[dict]]:
    """Returns (full_plain_text, batchUpdate requests) for a Docs API document body."""
    requests: List[dict] = []
    cursor = 1  # Docs body starts at index 1

    def insert_paragraph(text: str) -> Tuple[int, int]:
        nonlocal cursor, requests
        # Always end with newline to create a paragraph
        content = (text or "") + "\n"
        start = cursor
        requests.append({
            "insertText": {
                "location": {"index": cursor},
                "text": content
            }
        })
        cursor += len(content)
        end = cursor
        return start, end

    def set_paragraph_named_style(start: int, end: int, style: str):
        requests.append({
            "updateParagraphStyle": {
                "range": {"startIndex": start, "endIndex": end},
                "paragraphStyle": {"namedStyleType": style},
                "fields": "namedStyleType"
            }
        })

    def add_bullets(start: int, end: int, preset: str):
        requests.append({
            "createParagraphBullets": {
                "range": {"startIndex": start, "endIndex": end},
                "bulletPreset": preset
            }
        })

    def set_indent(start: int, end: int, level: int):
        # 36pt ~= 0.5 inch; tweak as desired for nesting
        indent_pt = level * 36
        requests.append({
            "updateParagraphStyle": {
                "range": {"startIndex": start, "endIndex": end},
                "paragraphStyle": {
                    "indentStart": {"magnitude": indent_pt, "unit": "PT"}
                },
                "fields": "indentStart"
            }
        })

    def style_mentions(start: int, text: str):
        # Apply bold + blue color to @mentions within this paragraph
        for m in MENTION_RE.finditer(text):
            s = start + m.start()
            e = start + m.end()
            requests.append({
                "updateTextStyle": {
                    "range": {"startIndex": s, "endIndex": e},
                    "textStyle": {
                        "bold": True,
                        "foregroundColor": {
                            "color": {"rgbColor": {"blue": 0.8, "red": 0.1, "green": 0.2}}
                        }
                    },
                    "fields": "bold,foregroundColor"
                }
            })

    def style_footer(start: int, end: int):
        requests.append({
            "updateTextStyle": {
                "range": {"startIndex": start, "endIndex": end-1},  # exclude newline
                "textStyle": {
                    "italic": True,
                    "fontSize": {"magnitude": 10, "unit": "PT"},
                    "foregroundColor": {
                        "color": {"rgbColor": {"red": 0.45, "green": 0.45, "blue": 0.45}}
                    }
                },
                "fields": "italic,fontSize,foregroundColor"
            }
        })

    # Build content
    for b in blocks:
        if b.kind in ("h1", "h2", "h3", "p", "footer"):
            start, end = insert_paragraph(b.text)
            if b.kind == "h1":
                set_paragraph_named_style(start, end, "HEADING_1")
            elif b.kind == "h2":
                set_paragraph_named_style(start, end, "HEADING_2")
            elif b.kind == "h3":
                set_paragraph_named_style(start, end, "HEADING_3")
            elif b.kind == "footer":
                style_footer(start, end)

            # Mention styling in normal paragraphs + bullets + footer too
            style_mentions(start, b.text)

        elif b.kind == "bullet":
            start, end = insert_paragraph(b.text)
            preset = "BULLET_DISC_CIRCLE_SQUARE"
            if b.checkbox:
                preset = "BULLET_CHECKBOX" if not b.checked else "BULLET_CHECKBOX_CHECKED"
            add_bullets(start, end, preset)
            if b.level > 0:
                set_indent(start, end, b.level)
            style_mentions(start, b.text)

        elif b.kind == "hr":
            # Add a blank line + a horizontal separator effect using a styled line of em dashes
            start, end = insert_paragraph("—" * 24)
            requests.append({
                "updateTextStyle": {
                    "range": {"startIndex": start, "endIndex": end-1},
                    "textStyle": {
                        "foregroundColor": {
                            "color": {"rgbColor": {"red": 0.7, "green": 0.7, "blue": 0.7}}
                        }
                    },
                    "fields": "foregroundColor"
                }
            })
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": start, "endIndex": end},
                    "paragraphStyle": {"alignment": "CENTER"},
                    "fields": "alignment"
                }
            })
        else:
            raise ValueError(f"Unknown block kind: {b.kind}")

    # Remove the initial empty paragraph (the doc starts with a blank line)
    # We delete from index 1 to 2 (first newline) after our insertions; safest to keep as-is for simplicity.
    return "", requests

_, requests = build_doc_requests(blocks)
len(requests), requests[0]


In [ ]:
def create_google_doc_from_markdown(md: str, doc_title: str = "Product Team Sync (Imported)") -> str:
    """Creates a Google Doc and populates it with formatted content derived from markdown."""
    try:
        docs = build("docs", "v1")
    except Exception as e:
        raise RuntimeError(f"Failed to initialize Google Docs API client: {e}") from e

    blocks = parse_markdown(md)
    _, reqs = build_doc_requests(blocks)

    try:
        doc = docs.documents().create(body={"title": doc_title}).execute()
        doc_id = doc.get("documentId")
        if not doc_id:
            raise RuntimeError("Docs API did not return a documentId.")
    except HttpError as e:
        raise RuntimeError(f"Failed to create Google Doc (HTTP error): {e}") from e
    except Exception as e:
        raise RuntimeError(f"Failed to create Google Doc: {e}") from e

    try:
        # Apply all formatting + text insertion in one batchUpdate
        docs.documents().batchUpdate(documentId=doc_id, body={"requests": reqs}).execute()
    except HttpError as e:
        raise RuntimeError(f"Failed to write/format content (HTTP error): {e}") from e
    except Exception as e:
        raise RuntimeError(f"Failed to write/format content: {e}") from e

    return f"https://docs.google.com/document/d/{doc_id}/edit"

doc_url = create_google_doc_from_markdown(MARKDOWN_NOTES, doc_title="Product Team Sync - May 15, 2023")
print("✅ Created Google Doc:")
print(doc_url)
